In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
 # Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("StopWords").getOrCreate()

In [3]:
# Create sample Dataframe
sentenceData = spark.createDataFrame([
                                   (0, "My name is Amir ElTabakh"),
                                   (1, "I am a junior in college at this time"),
                                   (2, "I am interning at NASA")
], ["id", "raw"])

sentenceData.show()

+---+--------------------+
| id|                 raw|
+---+--------------------+
|  0|My name is Amir E...|
|  1|I am a junior in ...|
|  2|I am interning at...|
+---+--------------------+



In [4]:
# Import the Tokenizer library
from pyspark.ml.feature import Tokenizer

# Initialize tokenizer variable
tokenizer = Tokenizer(inputCol="raw", outputCol="words")

# Transform and show DataFrame
tokenized_df = tokenizer.transform(sentenceData)
tokenized_df.show(truncate=False)

+---+-------------------------------------+-----------------------------------------------+
|id |raw                                  |words                                          |
+---+-------------------------------------+-----------------------------------------------+
|0  |My name is Amir ElTabakh             |[my, name, is, amir, eltabakh]                 |
|1  |I am a junior in college at this time|[i, am, a, junior, in, college, at, this, time]|
|2  |I am interning at NASA               |[i, am, interning, at, nasa]                   |
+---+-------------------------------------+-----------------------------------------------+



In [5]:
# Import the StopWordsRemove library
from pyspark.ml.feature import StopWordsRemover

In [6]:
# Run the Remover
remover = StopWordsRemover(inputCol="words", outputCol="filtered")

The StopWordsRemover() function, which takes an input column that will be passed into the function, and an output column to add the results. This is stored in a variable for ease of use. The outcome shows the raw data, or input, and the result of running the StopWordsRemover() function. In the filtered column, all the stop words are removed and the result is displayed.

In [7]:
# Transform and show data
remover.transform(tokenized_df).show(truncate=False)

+---+-------------------------------------+-----------------------------------------------+-----------------------+
|id |raw                                  |words                                          |filtered               |
+---+-------------------------------------+-----------------------------------------------+-----------------------+
|0  |My name is Amir ElTabakh             |[my, name, is, amir, eltabakh]                 |[name, amir, eltabakh] |
|1  |I am a junior in college at this time|[i, am, a, junior, in, college, at, this, time]|[junior, college, time]|
|2  |I am interning at NASA               |[i, am, interning, at, nasa]                   |[interning, nasa]      |
+---+-------------------------------------+-----------------------------------------------+-----------------------+

